In [ ]:
#@title # INSTALL
#@markdown It may takes 1m 40s ~ 2m.

# if error occurs, remove > /dev/null 2>&1 and test
print("PYTHON...")
!apt-get install -y python3.10  > /dev/null 2>&1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1 > /dev/null 2>&1
!update-alternatives --set python3 /usr/bin/python3.10 > /dev/null 2>&1
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3 > /dev/null 2>&1
!pip install pip==23.3.1 > /dev/null 2>&1

!apt-get install -y ffmpeg > /dev/null 2>&1
print('FFMPEG...')

!pip install uv > /dev/null 2>&1
print('UV...')

!git clone https://github.com/tsyqax/TTS-RVC-AutoMaker.git DIR > /dev/null 2>&1
%cd DIR
print('GITHUB...')

print('INSTALLING...')

!uv pip install -r requirements.txt --system > /dev/null 2>&1
!uv pip install torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118 --system > /dev/null 2>&1
!python model_down.py

print('DONE!')

In [ ]:
#@title #**MODEL DOWNLOAD**
#@markdown Huggingface or Google Drive Link
url = "https://huggingface.co/jarari/RVC-v2/resolve/main/seele_kr.zip"  # @param {type:"string"}
#@markdown 모델 이름
dir_name = "seele_kr"  # @param {type:"string"}

import os
import urllib.request
import zipfile
import shutil
import gdown

#filename = f'{dir_name}.zip'
models_dir = os.path.join(os.getcwd(), 'models') # CURRENT / models /
download_file = os.path.join(models_dir, f'{dir_name}.zip') # CURRENT / models / dirname.zip
download_dir = os.path.join(models_dir, dir_name) # CURRENT / models / dir_name /
output = os.path.join(os.getcwd(), 'output') # CURRENT / output

os.makedirs(models_dir, exist_ok=True)

print('STARTING...!')

def unzip2(filename, output):
  os.makedirs(output, exist_ok=True)
  with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(output)
  os.remove(filename)

if 'huggingface.co' in url: # hugginface
  try:
    zipname = url.split('/')[-1]
    print("DOWNLOADING...!!")

    if os.path.exists(download_dir):
      print(f'RE---DOWNLOADING...!!')
      shutil.rmtree(download_dir)

    urllib.request.urlretrieve(url, download_file)

    print('UNZIPPING..!!!')
    unzip2(download_file, download_dir)

    print('DOWNLOADED!!')

  except Exception as e:
     raise Exception(str(e))

elif 'drive.google.com' in url:
  file_id = url.split('/')[-2]
  try:
    print("DOWNLOADING...!!")

    if os.path.exists(download_dir):
      print(f'RE---DOWNLOADING...!!')
      shutil.rmtree(download_dir)

    gdown.download(f'https://drive.google.com/uc?id={file_id}', download_file, quiet=False)
    unzip2(download_file, download_dir)
    print('DOWNLOADED!!')

  except Exception as e:
      raise Exception(str(e))
else:
  print('InValid LINK... HuggingFace or Google Drive is OKAY')

In [ ]:
#@title # **TTS FILE GENERATE**
#@markdown Use EdgeTTS (use gTTS when error).<br>gTTS can't select gender.<br>

#@markdown TTS Text
text = "hello hello heello"  # @param {type:"string"}

#@markdown Gender Select (Other = random)
tts_castle = "Female"  # @param ["Male", "Female", "Other"] {allow-input: false}

import random
if tts_castle == "Other":
  tts_castle = random.choice(["Male", "Female"])

#@markdown [Language]
tts_lang = "en"  # @param ["ko", "en", "ja"] {allow-input: false}

!python tts.py \
    --text "{text}" \
    --castle "{tts_castle}" \
    --language "{tts_lang}"

In [ ]:
#@title # **INFERENCE**

#@markdown Filename (Optional)<br>
#@markdown If it is blank, We will use TTS file which was generated above.
input = ""  # @param {type:"string"}

#@markdown RVC Name
dir_name = "seele_kr"  # @param {type:"string"}

#@markdown Vocal Pitch Change (Pork * 1.2 = semiton, 10 Pork = 1 Octarv)
pitch_vocal = 1.5  # @param {type:"slider", min:-15, max:15, step:0.5}

# @markdown Inference Algorithm
rvc_method = "fcpe"  # @param ["rmvpe", "fcpe"] {allow-input: false}

#@markdown Index Rate (Default 0.75)
rvc_index_rate = 0.75  # @param {type:"slider", min:0, max:1, step:0.05}

# Pass the defined variables as command-line arguments to main.py

'''if 'https://' not in input and 'http://' not in input:
  if '/content/' not in input:
    input = f'/content/drive/MyDrive/{input}'
  from google.colab import drive
  drive.mount('/content/drive')'''


!python main.py \
    --input "{input}" \
    --rvc-name "{dir_name}" \
    --pitch-vocal {pitch_vocal} \
    --rvc-method "{rvc_method}" \
    --index-rate {rvc_index_rate}